## Modelling

### Mount Google Collab

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Imports

In [30]:
import numpy as np
import pandas as pd

from imblearn.over_sampling import SMOTE
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

### Load Dataset

In [4]:
df = pd.read_csv("/content/drive/MyDrive/Datasets/cleaned_data.csv")
df.head()

,absolute_magnitude,estimated_diameter_min,estimated_diameter_max,relative_velocity,miss_distance,is_hazardous
0,19.14,0.394962,0.883161,71745.401048,5.814362e+07,0
1,18.50,0.530341,1.185878,109949.757148,5.580105e+07,1
2,21.45,0.136319,0.304818,24865.506798,6.720689e+07,0
3,20.63,0.198863,0.444672,78890.076805,3.039644e+07,0
4,22.70,0.076658,0.171412,56036.519484,6.311863e+07,0


## Handling Class Imbalance
SMOTE

In [5]:
new_df = df.drop(['estimated_diameter_min'], axis=1)
new_df.head()

,absolute_magnitude,estimated_diameter_max,relative_velocity,miss_distance,is_hazardous
0,19.14,0.883161,71745.401048,5.814362e+07,0
1,18.50,1.185878,109949.757148,5.580105e+07,1
2,21.45,0.304818,24865.506798,6.720689e+07,0
3,20.63,0.444672,78890.076805,3.039644e+07,0
4,22.70,0.171412,56036.519484,6.311863e+07,0


Transforming the values of the `miss_distance` column

In [6]:
new_df['miss_distance'] = np.log1p(new_df['miss_distance'])
new_df

,absolute_magnitude,estimated_diameter_max,relative_velocity,miss_distance,is_hazardous
0,19.140,0.883161,71745.401048,17.878427,0
1,18.500,1.185878,109949.757148,17.837303,1
2,21.450,0.304818,24865.506798,18.023286,0
3,20.630,0.444672,78890.076805,17.229836,0
4,22.700,0.171412,56036.519484,17.960526,0
...,...,...,...,...,...
338166,28.580,0.011430,56646.985988,17.975416,0
338167,28.690,0.010865,21130.768947,17.199522,0
338168,21.919,0.245607,11832.041031,17.794459,0
338169,23.887,0.099229,56198.382733,15.461231,0


In [7]:
# Unique values and their counts
unique, count = np.unique(new_df['is_hazardous'], return_counts=True)

# Percentage calculation
percentage = (count / count.sum()) * 100

# Display results
for u, c, p in zip(unique, count, percentage):
    print(f"Class: {u}, Count: {c}, Percentage: {p:.2f}%")

Class: 0, Count: 295009, Percentage: 87.24%
Class: 1, Count: 43162, Percentage: 12.76%


In [8]:
X = new_df.drop('is_hazardous', axis=1)
y = new_df['is_hazardous']

In [9]:
print(X.shape)
print(y.shape)

(338171, 4)
(338171,)


In [10]:
## Apply SMOTE
smote = SMOTE(random_state=42)
X_smote, y_smote = smote.fit_resample(X, y)

In [11]:
# Unique values and their counts in y_train_smote
unique, count = np.unique(y_smote, return_counts=True)

# Percentage calculation
percentage = (count / count.sum()) * 100

# Display results
for u, c, p in zip(unique, count, percentage):
    print(f"Class: {u}, Count: {c}, Percentage: {p:.2f}%")

Class: 0, Count: 295009, Percentage: 50.00%
Class: 1, Count: 295009, Percentage: 50.00%


In [24]:
X_train, X_test, y_train, y_test = train_test_split(X_smote, y_smote, test_size=0.2, random_state=42)

# Setup a new experiment

In [26]:
rf = RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='sqrt',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       monotonic_cst=None, n_estimators=100, n_jobs=-1,
                       oob_score=False, random_state=123, verbose=0,
                       warm_start=False)

### Cross Validation


In [27]:
scores = cross_val_score(rf, X_smote, y_smote, cv=10, scoring='accuracy')

In [28]:
print("Average Accuracy: ", np.mean(scores))

Average Accuracy:  0.9342292371032963


#### Final Fitting

In [29]:
 rf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='sqrt',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       monotonic_cst=None, n_estimators=100, n_jobs=-1,
                       oob_score=False, random_state=123, verbose=0,
                       warm_start=False)

#### Model Evaluation

In [31]:
y_pred = rf.predict(X_test)  # Make predictions
print(classification_report(y_test, y_pred))  # Evaluation metrics

              precision    recall  f1-score   support

           0       0.95      0.93      0.94     59040
           1       0.94      0.95      0.94     58964

    accuracy                           0.94    118004
   macro avg       0.94      0.94      0.94    118004
weighted avg       0.94      0.94      0.94    118004



In [32]:
print(f"Accuracy: {accuracy_score(y_test, y_pred):.2f}")

Accuracy: 0.94


#### Saving Model

In [33]:
%pip install skops

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.0/147.0 kB 3.0 MB/s eta 0:00:00


In [34]:
import skops.io as sio
obj = sio.dump(rf, "neo_rf.skops")

In [35]:
unknown_types = sio.get_untrusted_types(file="neo_rf.skops")
# investigate the contents of unknown_types, and only load if you trust
# everything you see.
clf = sio.load("neo_rf.skops", trusted=unknown_types)

In [36]:
print(clf)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='sqrt',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       monotonic_cst=None, n_estimators=100, n_jobs=-1,
                       oob_score=False, random_state=123, verbose=0,
                       warm_start=False)


In [37]:
y_pred = clf.predict(X_test)  # Make predictions
print(classification_report(y_test, y_pred))  # Evaluation metrics

              precision    recall  f1-score   support

           0       0.95      0.93      0.94     59040
           1       0.94      0.95      0.94     58964

    accuracy                           0.94    118004
   macro avg       0.94      0.94      0.94    118004
weighted avg       0.94      0.94      0.94    118004



In [38]:
print(f"Accuracy: {accuracy_score(y_test, y_pred):.2f}")

Accuracy: 0.94
